In [ ]:
!pip install python-binance

In [ ]:
pip install TA-Lib

In [ ]:
!pip install pandas_ta

In [ ]:
!pip install psutil

In [ ]:
!pip install matplotlib

In [ ]:
!pip install plotly

In [ ]:
!pip install ccxt

In [ ]:
!pip install binance

In [ ]:
info = client.futures_exchange_info()

In [ ]:
q = [i['symbol'] for i in info['symbols']]

In [ ]:
def get_supertrend(high, low, close, lookback, multiplier):
    
    # ATR
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.ewm(lookback).mean()
    
    # H/L AVG AND BASIC UPPER & LOWER BAND
    
    hl_avg = (high + low) / 2
    upper_band = (hl_avg + multiplier * atr).dropna()
    lower_band = (hl_avg - multiplier * atr).dropna()
    
    # FINAL UPPER BAND
    
    final_bands = pd.DataFrame(columns = ['upper', 'lower'])
    final_bands.iloc[:,0] = [x for x in upper_band - upper_band]
    final_bands.iloc[:,1] = final_bands.iloc[:,0]
    
    for i in range(len(final_bands)):
        if i == 0:
            final_bands.iloc[i,0] = 0
        else:
            if (upper_band[i] < final_bands.iloc[i-1,0]) | (close[i-1] > final_bands.iloc[i-1,0]):
                final_bands.iloc[i,0] = upper_band[i]
            else:
                final_bands.iloc[i,0] = final_bands.iloc[i-1,0]
    
    # FINAL LOWER BAND
    
    for i in range(len(final_bands)):
        if i == 0:
            final_bands.iloc[i, 1] = 0
        else:
            if (lower_band[i] > final_bands.iloc[i-1,1]) | (close[i-1] < final_bands.iloc[i-1,1]):
                final_bands.iloc[i,1] = lower_band[i]
            else:
                final_bands.iloc[i,1] = final_bands.iloc[i-1,1]
    
    # SUPERTREND
    
    supertrend = pd.DataFrame(columns = [f'supertrend_{lookback}'])
    supertrend.iloc[:,0] = [x for x in final_bands['upper'] - final_bands['upper']]
    
    for i in range(len(supertrend)):
        if i == 0:
            supertrend.iloc[i, 0] = 0
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 0] and close[i] < final_bands.iloc[i, 0]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 0]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 0] and close[i] > final_bands.iloc[i, 0]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 1]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 1] and close[i] > final_bands.iloc[i, 1]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 1]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 1] and close[i] < final_bands.iloc[i, 1]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 0]
    
    supertrend = supertrend.set_index(upper_band.index)
    supertrend = supertrend.dropna()[1:]
    
    # ST UPTREND/DOWNTREND
    
    upt = []
    dt = []
    close = close.iloc[len(close) - len(supertrend):]

    for i in range(len(supertrend)):
        if close[i] > supertrend.iloc[i, 0]:
            upt.append(supertrend.iloc[i, 0])
            dt.append(np.nan)
        elif close[i] < supertrend.iloc[i, 0]:
            upt.append(np.nan)
            dt.append(supertrend.iloc[i, 0])
        else:
            upt.append(np.nan)
            dt.append(np.nan)
            
    st, upt, dt = pd.Series(supertrend.iloc[:, 0]), pd.Series(upt), pd.Series(dt)
    upt.index, dt.index = supertrend.index, supertrend.index
    
    return st, upt, dt

In [ ]:
client.futures_historical_klines(
    symbol=i,
    interval='30m',  # can play with this e.g. '1h', '4h', '1w', etc.
    start_str='2021-06-01',
    end_str='2021-12-18'
    )

In [1]:
from binance import Client
import pandas as pd
import talib as ta
import pandas_ta as pta
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
import psutil
import collections
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import time

In [2]:
while True:
    df15 = get_15m_data(symbol)

    df30 = get_30m_data(symbol)

    fig1 = go.Figure()

    fig1 = make_subplots(
        rows=1, cols=2,
        column_widths=[0.6, 0.4])

    fig1.add_trace(go.Candlestick(x=df30.index[-5:],
                    open=df30['Open'][-5:],
                    high=df30['High'][-5:],
                    low=df30['Low'][-5:],
                    close=df30['Close'][-5:], showlegend=False),row=1,col=1)
    fig1.add_trace(
        go.Scatter(
            x=df30.index[-5:],
            y=df30.Stval[-5:], showlegend=False, line_color="#00FF00" if df30.Stval[-1] < df30.Close[-1] else "#FF0000"
        ),row=1,col=1)
    fig1.add_trace(
        go.Scatter(
            mode="markers+text",
            x=[df30.index[-1]],
            y=[df30.Close[-1]], showlegend=False,text=[f"{df30.Close[-1]}"],textposition="top center"
        ),row=1,col=1)

    fig1.add_trace(
        go.Scatter(x=df15.index[-5:], y=df15.BBP[-5:],showlegend=False),
        row=1, col=2
    )
    fig1.add_trace(
        go.Scatter(x=df15.index[-5:], y=df15.upper[-5:],showlegend=False),
        row=1, col=2
    )
    fig1.add_trace(
        go.Scatter(x=df15.index[-5:], y=df15.lower[-5:],showlegend=False),
        row=1, col=2
    )
    fig1.write_html(r"C:\Users\mahes\OneDrive\Documents\Full Stack Web Development\Binance\hello.html")
    time.sleep(25)

NameError: name 'get_15m_data' is not defined

In [3]:
api_key = '3QCKHUvnmZBo8PGBxNmIx2n7kuTTYvobECbfksrptBG9FC6YU5l9FrLjqJ9KgfOa'
api_secret = 'hwlBEnfgVUIj8m6wPHAv7TW6Y3KgAyhP6WAIoV0cRsJUWrhIsVVZP8s3p4CULfWs'
client = Client(api_key, api_secret)

In [4]:
[i for i in client.futures_account_balance() if float(i['balance'])!=0]

[{'accountAlias': 'FzFzAuuXXqXquXmY',
  'asset': 'USDT',
  'balance': '135.59269006',
  'withdrawAvailable': '135.59269006',
  'updateTime': 1640280359183}]

In [5]:
def heiken_ashi(df):
    df_ha = df.copy()
    for i in range(df_ha.shape[0]):
        if i > 0:
            df_ha.loc[df_ha.index[i],'Open'] = (df['Open'][i-1] + df['Close'][i-1])/2

        df_ha.loc[df_ha.index[i],'Close'] = (df['Open'][i] + df['Close'][i] + df['Low'][i] +  df['High'][i])/4
    df_ha = df_ha.iloc[1:,:]
    return df_ha

In [45]:
def get_15m_data(symbol):
#   Binance Client
    frame = pd.DataFrame(client.futures_historical_klines(symbol,'15m',f'{58} h ago UTC'))
    frame = frame.iloc[:,:5]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index,unit='ms')
    frame = frame.astype(float)
    frame = heiken_ashi(frame)
    bbp=BBP(frame)
    frame.insert(loc = len(frame.columns), column='BBP',value=bbp)
    frame.index = frame.index + pd.DateOffset(hours=5.5)
    frame['upper'] = 1
    frame['lower'] = 0
    return frame

In [46]:
def get_30m_data(symbol):
#   Binance Client
    frame = pd.DataFrame(client.futures_historical_klines(symbol,'30m',f'{58} h ago UTC'))
    frame = frame.iloc[:,:5]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index,unit='ms')
    frame = frame.astype(float)
    frame.index = frame.index + pd.DateOffset(hours=5.5)
    up, mid, low = ta.BBANDS(frame.Close.values, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    supertrend = pta.supertrend(frame['High'], frame['Low'],frame['Close'],5,1.5)["SUPERTd_5_1.5"]
    strendvals = pta.supertrend(frame['High'], frame['Low'],frame['Close'],5,1.5).iloc[:,0]
    frame.insert(loc=len(frame.columns), column='Supertrend', value = supertrend)
    frame.insert(loc=len(frame.columns), column='Stval', value = strendvals)
# Supertrend Alt
#     frame['st'],frame['st_ut'],frame['st_dt'] = get_supertrend(frame['High'], frame['Low'], frame['Close'], 5, 1.5)
    frame.insert(loc=len(frame.columns), column='up', value = up)
    frame.insert(loc=len(frame.columns), column='down', value = low)
    return frame

In [8]:
def BBP(price):
    up, mid, low = ta.BBANDS(price.Close.values, timeperiod=200, nbdevup=1, nbdevdn=1, matype=0)
    bbp = (price['Close'] - low) / (up - low)
    return bbp

In [9]:
def ceil_dt(now):
    return now - (now - datetime.min) % timedelta(minutes=30)

In [60]:
symbol = 'ETHUSDT'
leverage = 20
balance_percent = 98

df15 = get_15m_data(symbol)

df30 = get_30m_data(symbol)
# in_trade = False
# long_entry = 0
# short_entry = 0
# long_exit = 0
# short_exit = 0
# init_balance = 100
# balance = init_balance
# leverage = 20
# sl = 0
# fee = 0.04
# tp = 0
# position = 'long'
# position_size = []
# trades = []
trade_dir = {}
trade_times= []
for i in range(2,len(df15)):
    if (df15.BBP[i-1]<-0.0001 and df15.BBP[i]>0.0001) or (df15.BBP[i-1]<0.9999 and df15.BBP[i]>1.0001):
        x = str(df15.index[i])
        if x[-5:-3] not in ['30','00']:
            if x[-5:-3] == '15':
                x = x[:-5]+'00'+x[-3:]
            else:
                x = x[:-5]+'30'+x[-3:]
        if df30.loc[x].High > df30.loc[x].up:
            if (abs(df30.loc[x].Close-df30.loc[x].Open)/df30.loc[x].Open) > 0.02:
                incremental_factor = 4
            else:
                incremental_factor = 2
            x = str(df15.index[i+incremental_factor])
            if x[-5:-3] not in ['30','00']:
                if x[-5:-3] == '15':
                    x = x[:-5]+'30'+x[-3:]
                else:
                    x = x[:-5]+'00'+x[-3:]
        if (df30.loc[x]['Supertrend'] ==1)  and (abs(df30.loc[x].Close-df30.loc[x].Open)/df30.loc[x].Open)<.021:
            trade_dir[x] = 'Up'
            trade_times.append(x)
    elif ((df15.BBP[i-1]>-0.0001 and df15.BBP[i]<.0001) or (df15.BBP[i-1]>0.9999 and df15.BBP[i]<1.0001)):
        x = str(df15.index[i])
        if x[-5:-3] not in ['30','00']:
            if x[-5:-3] == '15':
                x = x[:-5]+'00'+x[-3:]
            else:
                x = x[:-5]+'30'+x[-3:]
        if df30.loc[x].Low < df30.loc[x].down:
            if (abs(df30.loc[x].Close-df30.loc[x].Open)/df30.loc[x].Open) > 0.02:
                incremental_factor = 4
            else:
                incremental_factor = 2
            x = str(df15.index[i+incremental_factor])
            if x[-5:-3] not in ['30','00']:
                if x[-5:-3] == '15':
                    x = x[:-5]+'30'+x[-3:]
                else:
                    x = x[:-5]+'00'+x[-3:]
        if (df30.loc[x]['Supertrend'] == -1)  and (abs(df30.loc[x].Close-df30.loc[x].Open)/df30.loc[x].Open)<.021:
            trade_dir[x] = 'Down'
            trade_times.append(x)

In [61]:
trade_dir

{}

In [40]:
in_trade = False
init_balance = 96.55
balance = init_balance
long_entry = 0
long_exit = 0
short_entry = 0
short_exit = 0
tp = 0
sl = 0
mode = ''
position_size = 0
fee = 0.0004
leverage = 20
pnl = 0
trades_obj = []
df30['B_S'] = ''
entry_time = 0
exit_time = 0
pl = []
for i in df30.index:
    if balance <= 0:
        print('Ended due to zero balance')
        break
    invest = 100 if balance>100 else balance #balance if balance*leverage<6000 else 6000/20
    if not in_trade:
        if str(i) in trade_times:
            if trade_dir[str(i)] == 'Up':
                print('Enter Long at ', str(i))
                mode = 'Long'
                long_entry = df30.loc[i].Close
                tp = long_entry*1.013
                sl = long_entry*0.983
                position_size = invest*leverage*(1-fee)/long_entry
                in_trade = True
                df30.loc[i].B_S = "Long_Entry"
                entry_time = str(i)
                continue
            else:
                print('Enter Short at ',str(i))
                mode = 'Short'
                short_entry = df30.loc[i].Close
                tp = short_entry*0.987
                sl = short_entry*1.017
                in_trade = True
                df30.loc[i].B_S = "Short_Entry"
                entry_time = str(i)
                continue
    if in_trade:
        if mode == 'Long':
            if df30.loc[i].High >= tp  or df30.loc[i].Low <= sl:
                df30.loc[i].B_S = "Long_Exit"
                print('Exited Long')
                long_exit = tp if df30.loc[i].High >= tp else sl
                pnl = position_size*long_exit*(1-fee) - invest*leverage*(1-fee)
                balance += pnl
                exit_time = str(i)
                trades_obj.append({
                    "Initial Balance": init_balance,
                    "long_entry": long_entry,
                    "Filled": position_size,
                    "long_exit": long_exit,
                    "tp": tp,
                    "sl": sl,
                    "Sell mode":  'Profit' if pnl>0 else "Loss",
                    "PnL": pnl,
                    "Balance": balance,
                    "invested" : invest,
                    "Entry time": entry_time,
                    "Exit time": exit_time
                })
                pl.append(pnl)
                long_entry = 0
                long_exit = 0
                tp = 0
                sl = 0
                pnl = 0
                position_size = 0
                in_trade = False
                exit_time = 0
                entry_time = 0
        else:
            if df30.loc[i].Low <= tp or df30.loc[i].High >= sl:
                df30.loc[i].B_S = "Short_Exit"
                print('Exited Short')
                short_exit = tp
                position_size = invest*leverage*(1-fee)/short_exit
                pnl = position_size*short_entry*(1-fee) - invest*leverage*(1-fee)
                balance += pnl
                exit_time = str(i)
                trades_obj.append({
                    "Initial Balance": init_balance,
                    "short_entry": short_entry,
                    "Filled": position_size,
                    "short_exit": short_exit,
                    "tp": tp,
                    "sl": sl,
                    "Sell mode":  'Profit' if pnl>0 else "Loss",
                    "PnL": pnl,
                    "Balance": balance,
                    "invested" : invest,
                    "Entry time": entry_time,
                    "Exit time": exit_time
                    
                })
                pl.append(pnl)
                in_trade = False
                short_entry = 0
                short_exit = 0
                tp = 0
                sl = 0
                pnl = 0
                position_size = 0
                exit_time = 0
                entry_time = 0

Enter Short at  2021-05-15 07:00:00
Exited Short
Enter Short at  2021-05-15 08:00:00
Exited Short
Enter Short at  2021-05-15 18:30:00
Exited Short
Enter Long at  2021-05-16 12:00:00
Exited Long
Enter Long at  2021-05-16 16:00:00
Exited Long
Enter Short at  2021-05-16 18:00:00
Exited Short
Enter Short at  2021-05-16 20:00:00
Exited Short
Enter Short at  2021-05-17 23:00:00
Exited Short
Enter Long at  2021-05-18 16:00:00
Exited Long
Enter Short at  2021-05-21 18:30:00
Exited Short
Enter Long at  2021-05-22 18:30:00
Exited Long
Enter Short at  2021-05-22 21:30:00
Exited Short
Enter Short at  2021-05-25 11:00:00
Exited Short
Enter Long at  2021-05-26 12:30:00
Exited Long
Enter Short at  2021-05-27 01:30:00
Exited Short
Enter Short at  2021-05-27 07:30:00
Exited Short
Enter Long at  2021-05-27 14:30:00
Exited Long
Enter Long at  2021-05-27 16:30:00
Exited Long
Enter Long at  2021-05-27 19:30:00
Exited Long
Enter Long at  2021-05-29 07:30:00
Exited Long
Enter Short at  2021-05-29 14:00:00
Ex

C:\Users\mahes\AppData\Roaming\Python\Python37\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()



Enter Short at  2021-07-10 23:00:00
Exited Short
Enter Long at  2021-07-11 16:30:00
Exited Long
Enter Long at  2021-07-12 03:00:00
Exited Long
Enter Short at  2021-07-12 21:30:00
Exited Short
Enter Long at  2021-07-13 05:00:00
Exited Long
Enter Long at  2021-07-14 15:00:00
Exited Long
Enter Short at  2021-07-15 12:30:00
Exited Short
Enter Long at  2021-07-15 19:00:00
Exited Long
Enter Short at  2021-07-16 01:00:00
Exited Short
Enter Long at  2021-07-16 21:00:00
Exited Long
Enter Long at  2021-07-17 00:00:00
Exited Long
Enter Long at  2021-07-17 06:30:00
Exited Long
Enter Long at  2021-07-17 14:00:00
Exited Long
Enter Long at  2021-07-18 00:30:00
Exited Long
Enter Long at  2021-07-18 06:30:00
Exited Long
Enter Short at  2021-07-18 20:00:00
Exited Short
Enter Long at  2021-07-20 22:00:00
Exited Long
Enter Long at  2021-07-21 05:00:00
Exited Long
Enter Long at  2021-07-21 15:00:00
Exited Long
Enter Short at  2021-07-22 01:30:00
Exited Short
Enter Short at  2021-07-22 08:00:00
Exited Shor

Enter Short at  2021-11-26 15:30:00
Exited Short
Enter Short at  2021-11-26 18:30:00
Exited Short
Enter Short at  2021-11-26 21:30:00
Exited Short
Enter Short at  2021-11-28 08:00:00
Exited Short
Enter Long at  2021-11-28 15:00:00
Exited Long
Enter Long at  2021-11-28 18:00:00
Exited Long
Enter Long at  2021-11-28 20:30:00
Exited Long
Enter Long at  2021-11-29 02:00:00
Exited Long
Enter Long at  2021-11-29 06:30:00
Exited Long
Enter Short at  2021-11-29 15:30:00
Exited Short
Enter Long at  2021-11-29 23:00:00
Exited Long
Enter Short at  2021-12-01 00:00:00
Exited Short
Enter Short at  2021-12-01 02:30:00
Exited Short
Enter Short at  2021-12-01 04:30:00
Exited Short
Enter Short at  2021-12-02 03:00:00
Exited Short
Enter Long at  2021-12-02 21:00:00
Exited Long
Enter Short at  2021-12-03 05:00:00
Exited Short
Enter Short at  2021-12-03 22:30:00
Exited Short
Enter Short at  2021-12-04 00:00:00
Exited Short
Enter Long at  2021-12-04 21:30:00
Exited Long
Enter Long at  2021-12-04 23:00:00
E

In [41]:
trades_obj

[{'Initial Balance': 96.55,
  'short_entry': 0.018273,
  'Filled': 107024.08273571868,
  'short_exit': 0.018035451,
  'tp': 0.018035451,
  'sl': 0.018583640999999998,
  'Sell mode': 'Profit',
  'PnL': 24.64120340425552,
  'Balance': 121.19120340425552,
  'invested': 96.55,
  'Entry time': '2021-05-15 07:00:00',
  'Exit time': '2021-05-15 07:30:00'},
 {'Initial Balance': 96.55,
  'short_entry': 0.017747,
  'Filled': 114133.76429219684,
  'short_exit': 0.017516288999999997,
  'tp': 0.017516288999999997,
  'sl': 0.018048698999999998,
  'Sell mode': 'Profit',
  'PnL': 25.521702127659637,
  'Balance': 146.71290553191517,
  'invested': 100,
  'Entry time': '2021-05-15 08:00:00',
  'Exit time': '2021-05-15 08:30:00'},
 {'Initial Balance': 96.55,
  'short_entry': 0.01744,
  'Filled': 116142.88502830373,
  'short_exit': 0.01721328,
  'tp': 0.01721328,
  'sl': 0.01773648,
  'Sell mode': 'Profit',
  'PnL': 25.521702127659637,
  'Balance': 172.2346076595748,
  'invested': 100,
  'Entry time': '202

In [42]:
pl

[24.64120340425552,
 25.521702127659637,
 25.521702127659637,
 -34.772485439999855,
 25.179524160000028,
 25.52170212765941,
 25.52170212765941,
 25.521702127659637,
 -34.772485439999855,
 25.521702127659864,
 25.179524160000028,
 25.521702127659637,
 25.521702127659864,
 -34.77248543999963,
 25.521702127659637,
 25.521702127659864,
 -34.77248544000008,
 -34.772485439999855,
 -34.77248543999963,
 25.179524160000256,
 25.521702127659864,
 25.521702127659637,
 25.521702127659864,
 -34.77248544000008,
 -34.77248543999963,
 -34.772485439999855,
 25.521702127659637,
 25.1795241599998,
 25.179524160000028,
 25.1795241599998,
 -34.77248543999963,
 25.179524160000028,
 25.52170212765941,
 25.521702127659637,
 -34.772485439999855,
 25.521702127659637,
 25.52170212765941,
 -34.77248544000008,
 -34.772485439999855,
 25.179524160000256,
 25.52170212765941,
 25.521702127659637,
 -34.77248544000008,
 -34.772485439999855,
 25.52170212765941,
 25.52170212765941,
 25.179524160000256,
 -34.7724854399998

In [43]:
sum(pl)*80

395716.94656217936

In [44]:
len([i for i in pl if i > 0])/len(pl)

0.7611607142857143